In [1]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [4]:
async def get_expiry(f_today, index):

    if index == 'bnf':    
        if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
            f_expiry = dt.date(2024, 1, 25)
        elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
            f_expiry = dt.date(2024, 1, 31)
        elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
            f_expiry = dt.date(2024, 2, 29)
        elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
            f_expiry = dt.date(2024, 2, 27)
        elif f_today < dt.date(2023, 9, 1):
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
        elif f_today >= dt.date(2023, 9, 1):
            if f_today.day < 24:
                days_to_wednesday = (2 - f_today.weekday()) % 7
                nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
                f_expiry = nearest_wednesday
                if nse.valid_days(
                    start_date=nearest_wednesday, end_date=nearest_wednesday
                ).empty:
                    f_expiry = nearest_wednesday - dt.timedelta(days=1)
            else:
                days_to_thursday = (3 - f_today.weekday()) % 7
                nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
                f_expiry = nearest_thursday
                if nse.valid_days(
                    start_date=nearest_thursday, end_date=nearest_thursday
                ).empty:
                    f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'nifty':
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'finnifty' or index == 'fnf':
        days_to_thursday = (1 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'midcpnifty' or index == 'midcp':
        days_to_thursday = (0 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

async def get_expiry_nifty(f_today):

    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"


def get_option_contract_name2(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [11]:
bnf_1min = pd.read_csv("../data/nifty_wave.csv")
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["time"])
bnf_1min = bnf_1min[bnf_1min["datetime"].dt.year >= 2017]

In [12]:
bnf_1min.tail()

,time,open,high,low,close,MA,Plot,Zero line,MACD Signal,datetime
17188,2024-12-05T09:15:00+05:30,24512.2000,24539.9500,24396.2000,24400.1000,24258.1475,59.4254,0,59.4254,2024-12-05 09:15:00+05:30
17189,2024-12-05T10:15:00+05:30,24400.3500,24441.7500,24295.5500,24368.0500,24259.3125,33.7507,0,33.7507,2024-12-05 10:15:00+05:30
17190,2024-12-05T11:15:00+05:30,24367.6000,24604.9000,24330.1000,24589.9000,24266.8637,70.0144,0,70.0144,2024-12-05 11:15:00+05:30
17191,2024-12-05T12:15:00+05:30,24589.7000,24648.6000,24560.7500,24631.1000,24275.7887,99.4746,0,99.4746,2024-12-05 12:15:00+05:30
17192,2024-12-05T13:15:00+05:30,24633.8500,24677.2000,24570.7500,24650.5000,24285.1175,118.7053,0,118.7053,2024-12-05 13:15:00+05:30


In [13]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [14]:
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"])
list_of_traded_dates = set(bnf_1min["datetime"].dt.date)
list_of_traded_dates

{datetime.date(2020, 9, 28),
 datetime.date(2018, 6, 25),
 datetime.date(2023, 5, 12),
 datetime.date(2023, 8, 28),
 datetime.date(2023, 12, 13),
 datetime.date(2023, 7, 10),
 datetime.date(2022, 1, 5),
 datetime.date(2022, 2, 15),
 datetime.date(2019, 9, 26),
 datetime.date(2023, 4, 27),
 datetime.date(2023, 9, 11),
 datetime.date(2020, 5, 29),
 datetime.date(2019, 2, 6),
 datetime.date(2022, 2, 4),
 datetime.date(2017, 4, 19),
 datetime.date(2020, 2, 5),
 datetime.date(2024, 3, 1),
 datetime.date(2023, 11, 1),
 datetime.date(2018, 10, 10),
 datetime.date(2023, 11, 3),
 datetime.date(2020, 4, 1),
 datetime.date(2021, 1, 21),
 datetime.date(2024, 4, 8),
 datetime.date(2024, 3, 19),
 datetime.date(2019, 9, 17),
 datetime.date(2022, 9, 30),
 datetime.date(2021, 4, 7),
 datetime.date(2022, 8, 11),
 datetime.date(2024, 6, 20),
 datetime.date(2021, 8, 20),
 datetime.date(2023, 1, 23),
 datetime.date(2023, 5, 29),
 datetime.date(2022, 12, 16),
 datetime.date(2017, 6, 23),
 datetime.date(2017

In [30]:
import pandas as pd

def calculate_signals(df, n, rsi_period, rsi_overbought, ema_length):
    # Calculate RSI
    def calculate_rsi(series, period):
        # Calculate the price changes
        delta = series.diff()
    
        # Separate gains and losses
        gain = delta.where(delta > 0, 0)
        loss = -delta.where(delta < 0, 0)
    
        # Calculate the first average gain and loss (Simple Moving Average for the first period)
        avg_gain = gain.rolling(window=period, min_periods=period).mean()
        avg_loss = loss.rolling(window=period, min_periods=period).mean()
    
        # Calculate the subsequent smoothed averages (similar to EMA)
        for i in range(period, len(series)):
            avg_gain.iloc[i] = (avg_gain.iloc[i - 1] * (period - 1) + gain.iloc[i]) / period
            avg_loss.iloc[i] = (avg_loss.iloc[i - 1] * (period - 1) + loss.iloc[i]) / period
    
        # Calculate the Relative Strength (RS) and the RSI
        rs = avg_gain / avg_loss
        rsi = 100 - (100 / (1 + rs))
    
        return rsi

    # Apply the TradingView-style RSI calculation
    df['RSI'] = calculate_rsi(df['close'], rsi_period)

    # Calculate the previous n candles' low
    df['Prev_N_Low'] = df['low'].rolling(window=n).min().shift(1)
    df['ema'] = df['close'].ewm(span=ema_length, adjust=False).mean()

    # Signal generation logic
    df['Sell_Signal'] = (
        (df['close'] < df['Prev_N_Low']) &
        (df['RSI'] > rsi_overbought)
    )

    return df

In [31]:
bnf_1min.head()

,time,open,high,low,close,MA,Plot,Zero line,MACD Signal,datetime,RSI,Prev_N_Low,ema,Sell_Signal
3497,2017-01-02T09:15:00+05:30,8212.0000,8212.0000,8141.0000,8154.8000,8037.0038,51.3767,0,51.3767,2017-01-02 09:15:00+05:30,NaN,NaN,8154.8000,False
3498,2017-01-02T10:15:00+05:30,8155.1500,8161.8000,8133.8000,8146.8000,8040.7650,41.7773,0,41.7773,2017-01-02 10:15:00+05:30,NaN,NaN,8153.7333,False
3499,2017-01-02T11:15:00+05:30,8146.9500,8159.0500,8137.5000,8150.3000,8044.7463,35.4942,0,35.4942,2017-01-02 11:15:00+05:30,NaN,NaN,8153.2756,False
3500,2017-01-02T12:15:00+05:30,8150.8500,8166.5500,8144.9500,8165.3500,8049.3050,34.1609,0,34.1609,2017-01-02 12:15:00+05:30,NaN,8133.8000,8154.8855,False
3501,2017-01-02T13:15:00+05:30,8165.8000,8195.6000,8160.8000,8192.4500,8054.3700,38.7247,0,38.7247,2017-01-02 13:15:00+05:30,85.0885,8133.8000,8159.8941,False


In [32]:
lev_ = 8
portfolio_value = 10000000
lot_size_ = 75
slippage_ = 0.0002

In [33]:
def backtest_short(df, n, rsi_period, rsi_overbought, ema_length):
    df = df.reset_index(drop=True)
    trade_book = []
    in_trade = False
    points = 0
    is_trailing_active = False
    trailing_sl = 0
    df = calculate_signals(df, n, rsi_period, rsi_overbought, ema_length)
    # print(df[df['Sell_Signal']].to_string())
    for i in range(n, len(df)):
        # print(i, df.loc[i], df.loc[i, 'open'])
        # break
        if df.loc[i, 'Sell_Signal'] and not in_trade:
            # Entry Triggered
            entry_price = df.loc[i, 'close']
            entry_time = df.loc[i, 'datetime']
            initial_sl = df.loc[i-n:i, 'high'].max()
            sl_in_points = initial_sl - entry_price
            target = 5 * sl_in_points
            in_trade = True

        if in_trade:
            if df.loc[i, 'high'] < df.loc[i, 'ema']:
                is_trailing_active = True
            if is_trailing_active:
                trailing_sl = df.loc[i, 'ema']
            # print(df.loc[i, 'open'], initial_sl)
            # break
            if df.loc[i, 'open'] > initial_sl and not points:
                # Gap condition
                if df.loc[i, 'close'] < initial_sl:
                    initial_sl = df.loc[i, 'high']
                else:
                    exit_price = df.loc[i, 'close']
                    exit_time = df.loc[i, 'datetime']
                    in_trade = False
                    remark = 'Gap SL'
                    points = entry_price - exit_price
            if df.loc[i, 'high'] > initial_sl and not points:
                # Initial SL Hit
                exit_price = df.loc[i, 'close']
                exit_time = df.loc[i, 'datetime']
                in_trade = False
                remark = 'Initial SL'
                points = entry_price - exit_price
            if is_trailing_active:
                if df.loc[i, 'close'] > trailing_sl and not points:
                    # TSL Hit
                    exit_price = df.loc[i, 'close']
                    exit_time = df.loc[i, 'datetime']
                    in_trade = False
                    remark = 'Trailing SL'
                    points = entry_price - exit_price
            if df.loc[i, 'low'] < entry_price - target and not points:
                # Target Hit
                exit_price = entry_price - target
                exit_time = df.loc[i, 'datetime']
                in_trade = False
                remark = 'Target'
                points = entry_price - exit_price

        if points:
            index_lev = lev_
            qty = int(round(portfolio_value * index_lev / entry_price / lot_size_)) * lot_size_
            slippage = slippage_ * (entry_price + exit_price)
            final_points = points - slippage
            trade = {
                "Trade Type": "Short",
                "Entry Time": entry_time,
                "Entry Price": entry_price,
                "Initial SL": initial_sl,
                "Trailing SL": trailing_sl,
                "Exit Time": exit_time,
                "Exit Price": exit_price,
                "Points Captured": points,
                "Slippage in Points": slippage,
                "After Costs": final_points,
                "PnL": final_points * qty,
                "Remarks": remark,
                "Qty": qty,
                "Leverage": index_lev,
                "ROI%": (final_points * qty / portfolio_value) * 100,
                "Trade Year": entry_time.year,
                "Trade Month": entry_time.month,
            }
            # print(trade)
            trade_book.append(trade)
            points = 0
            in_trade = False
            is_trailing_active = False
            trailing_sl = 0

    return pd.DataFrame(trade_book)
            

In [34]:
tb = backtest_short(bnf_1min, n=3, rsi_period=5, rsi_overbought=50, ema_length=14)

In [35]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2018, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2018, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation
    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    return {overall_roi_dd_ratio : stats_df8}

In [36]:
variation = 1
stats = generate_stats(tb, variation)
for x, y in stats.items():
    final_stats = y

final_stats

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2018,-11.1419,12,41.6667,2.7391,-3.5482,-14.4758,-0.7697,1
2019,7.6040,13,61.5385,3.5579,-4.1718,-14.6166,0.5202,1
2020,-40.9511,12,25.0000,4.1117,-5.9207,-35.4367,-1.1556,1
2021,13.4167,12,41.6667,8.8621,-4.4134,-10.7969,1.2426,1
2022,-4.2291,19,47.3684,5.0206,-4.9415,-34.9641,-0.1210,1
2023,-0.2985,11,36.3636,7.4521,-4.3010,-20.8245,-0.0143,1
2024,-17.4771,11,36.3636,3.7384,-4.6329,-28.5400,-0.6124,1
Overall,-53.0771,90.0000,43.5644,4.5229,-4.7353,-85.8471,-0.6183,1.0000


In [37]:
tb[tb['Trade Year'] > 2023]

,Trade Type,Entry Time,Entry Price,Initial SL,Trailing SL,Exit Time,Exit Price,Points Captured,Slippage in Points,After Costs,PnL,Remarks,Qty,Leverage,ROI%,Trade Year,Trade Month
90,Short,2024-01-30 10:15:00+05:30,21681.4000,21813.0500,21613.8636,2024-01-31 10:15:00+05:30,21658.2000,23.2000,8.6679,14.5321,53405.3940,Trailing SL,3675,8,0.5341,2024,1
91,Short,2024-04-26 10:15:00+05:30,22515.9000,22625.9500,22472.3018,2024-04-29 09:15:00+05:30,22480.6000,35.3000,8.9993,26.3007,92709.9675,Trailing SL,3525,8,0.9271,2024,4
92,Short,2024-05-15 09:15:00+05:30,22189.6000,22330.0000,22205.3400,2024-05-16 14:15:00+05:30,22403.2500,-213.6500,8.9186,-222.5686,-801246.8520,Initial SL,3600,8,-8.0125,2024,5
93,Short,2024-05-21 09:15:00+05:30,22465.7000,22520.2500,0.0000,2024-05-21 11:15:00+05:30,22546.3000,-80.6000,9.0024,-89.6024,-315848.4600,Initial SL,3525,8,-3.1585,2024,5
94,Short,2024-06-28 12:15:00+05:30,24060.7500,24236.3500,0.0000,2024-07-03 09:15:00+05:30,24267.8500,-207.1000,9.6657,-216.7657,-715326.8760,Gap SL,3300,8,-7.1533,2024,6
95,Short,2024-07-29 12:15:00+05:30,24833.0500,24999.7500,0.0000,2024-08-01 09:15:00+05:30,25037.1000,-204.0500,9.9740,-214.0240,-690227.4967,Gap SL,3225,8,-6.9023,2024,7
96,Short,2024-08-20 15:15:00+05:30,24672.8500,24734.1500,0.0000,2024-08-21 10:15:00+05:30,24693.5000,-20.6500,9.8733,-30.5233,-98437.5458,Initial SL,3225,8,-0.9844,2024,8
97,Short,2024-08-27 14:15:00+05:30,25024.6000,25073.1000,0.0000,2024-08-28 11:15:00+05:30,25086.8000,-62.2000,10.0223,-72.2223,-232916.8530,Initial SL,3225,8,-2.3292,2024,8
98,Short,2024-09-27 12:15:00+05:30,26189.8500,26277.3500,25922.0217,2024-10-01 10:15:00+05:30,25752.3500,437.5000,10.3884,427.1116,1313368.0470,Target,3075,8,13.1337,2024,9
99,Short,2024-10-28 14:15:00+05:30,24343.9000,24492.6000,24308.9852,2024-10-29 12:15:00+05:30,24323.3000,20.6000,9.7334,10.8666,35859.6480,Trailing SL,3300,8,0.3586,2024,10


In [38]:
stats_dictionary = {}
rsi_lengths = [14, 9, 21, 6, 12, 16]
for i in range(2, 5):
    for j in rsi_lengths:
        for k in range(30, 81, 5):
            for l in range(5, 36, 5):
                variation = f'n {i}, rsi {j}, threshold {k}, ema {l}'
                print(variation)
                tb = backtest_short(bnf_1min, i, j, k, l)
                if len(tb) > 0:
                    stats = generate_stats(tb, variation)
                    for x, y in stats.items():
                        if x > 8:
                            final_stats = y
                            print(final_stats.to_string())
                            stats_dictionary[x] = y


n 2, rsi 14, threshold 30, ema 5
n 2, rsi 14, threshold 30, ema 10
n 2, rsi 14, threshold 30, ema 15
n 2, rsi 14, threshold 30, ema 20
n 2, rsi 14, threshold 30, ema 25
n 2, rsi 14, threshold 30, ema 30
n 2, rsi 14, threshold 30, ema 35
n 2, rsi 14, threshold 35, ema 5
n 2, rsi 14, threshold 35, ema 10
n 2, rsi 14, threshold 35, ema 15
n 2, rsi 14, threshold 35, ema 20
n 2, rsi 14, threshold 35, ema 25
n 2, rsi 14, threshold 35, ema 30
n 2, rsi 14, threshold 35, ema 35
n 2, rsi 14, threshold 40, ema 5
n 2, rsi 14, threshold 40, ema 10
n 2, rsi 14, threshold 40, ema 15
n 2, rsi 14, threshold 40, ema 20
n 2, rsi 14, threshold 40, ema 25
n 2, rsi 14, threshold 40, ema 30
n 2, rsi 14, threshold 40, ema 35
n 2, rsi 14, threshold 45, ema 5
n 2, rsi 14, threshold 45, ema 10
n 2, rsi 14, threshold 45, ema 15
n 2, rsi 14, threshold 45, ema 20
n 2, rsi 14, threshold 45, ema 25
n 2, rsi 14, threshold 45, ema 30
n 2, rsi 14, threshold 45, ema 35
n 2, rsi 14, threshold 50, ema 5
n 2, rsi 14, thresh

/tmp/ipykernel_37279/873016624.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


n 2, rsi 14, threshold 70, ema 10


/tmp/ipykernel_37279/873016624.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


n 2, rsi 14, threshold 70, ema 15


...

n 4, rsi 16, threshold 70, ema 30


/tmp/ipykernel_37279/873016624.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


n 4, rsi 16, threshold 70, ema 35


/tmp/ipykernel_37279/873016624.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  roi_dd_ratio = total_roi / abs(max_drawdown)


n 4, rsi 16, threshold 75, ema 5
n 4, rsi 16, threshold 75, ema 10
n 4, rsi 16, threshold 75, ema 15
n 4, rsi 16, threshold 75, ema 20
n 4, rsi 16, threshold 75, ema 25
n 4, rsi 16, threshold 75, ema 30
n 4, rsi 16, threshold 75, ema 35
n 4, rsi 16, threshold 80, ema 5
n 4, rsi 16, threshold 80, ema 10
n 4, rsi 16, threshold 80, ema 15
n 4, rsi 16, threshold 80, ema 20
n 4, rsi 16, threshold 80, ema 25
n 4, rsi 16, threshold 80, ema 30
n 4, rsi 16, threshold 80, ema 35


In [39]:
stats_dictionary

{inf:         Total ROI Total Trades Win Rate Avg Profit% per Trade  \
 2018       0.0000            0      NaN                   NaN   
 2019      -6.6692            1   0.0000                   NaN   
 2020       1.6246            1 100.0000                1.6246   
 2021       7.6344            1 100.0000                7.6344   
 2022       0.0000            0      NaN                   NaN   
 2023       0.0000            0      NaN                   NaN   
 2024       0.0000            0      NaN                   NaN   
 Overall    2.5897            3  66.6667                4.6295   
 
         Avg Loss% per Trade Max Drawdown ROI/DD Ratio  \
 2018                    NaN          NaN          NaN   
 2019                -6.6692       0.0000         -inf   
 2020                    NaN       0.0000          inf   
 2021                    NaN       0.0000          inf   
 2022                    NaN          NaN          NaN   
 2023                    NaN          NaN          